In [1]:
!pip -q install jsonlines pyldavis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 25.6 MB/s eta 0:00:00


In [2]:
!pip install textdistance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

#gensim
import gensim
from gensim import corpora

#Visualization
from spacy import displacy
import pyLDAvis.gensim_models
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import json
import jsonlines
from collections import Counter
from tqdm.notebook import tqdm
import math

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet', 'omw-1.4'])

#warning
import warnings 
#from PyPDF2 import PdfFileReader
from os.path import join
from tqdm.notebook import tqdm

import textdistance

tqdm.pandas()

warnings.filterwarnings('ignore')
SKILL_DB_DIR = 'data/source_data/SkillDB'
ACC_DIR = 'data/transformed_data/acceptability'
JO_OUT = 'data/transformed_data/job_offers_out'
SKILL_OUT = 'data/transformed_data/extracted_skills'

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [4]:
!python3 -q -m spacy download fr_core_news_md

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-28 10:07:04.437260: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-28 10:07:04.437391: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-28 10:07:04.437411: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [5]:
!python -m spacy download fr_core_news_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-28 10:07:40.848536: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-28 10:07:40.848705: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-28 10:07:40.848735: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [6]:
fr_nlp = spacy.load('fr_core_news_sm')  # download model first
# fr_core_news_sm

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [75]:
def write_json_lines(file_name,dict_data):
    json_string = json.dumps(dict_data)
    with open(file_name, 'a') as f:
        f.write(json_string+"\n")
        
def read_json_lines(file_name):
    lines = []
    with open(file_name) as file_in:
        for line in file_in:
            lines.append(json.loads(line))
    return lines

def get_clean_transformer_data(df):
    df = df.explode('en_skills').reset_index(drop=True)
    df['en_skills'] = df['en_skills'].str.replace("I'm sorry, but I don't know.", '')
    return df

def infer_(df, classfier):
    all_data = []
    for k,g in tqdm(df.groupby(np.arange(len(df))//2000)):
#         return g
        group_skills = list(g['en_skills'])
        ids = list(g['id'])
#         print(group_skills)
        all_classes = classfier(group_skills)
        for ix, i in enumerate(all_classes):
            all_classes[ix]['id'] = ids[ix]
        all_data += all_classes
    return all_data

def process_acceptability_df(acc_df):
    labels_series = pd.DataFrame(acc_df).groupby('id')['label'].apply(list)
    score_series = pd.DataFrame(acc_df).groupby('id')['score'].apply(list)
    return labels_series, score_series
    
def concat_tables_jo_ESCO(acc_transformers_df, acc_argos_df):
    labels_series, score_series = process_acceptability_df(acc_transformers_df)
    labels_series_ar, score_series_ar = process_acceptability_df(acc_argos_df)
    original_df = pd.DataFrame(read_json_lines(join(SKILL_OUT, 'fr_extracted_skills_v3.jsonl')))
    original_df = original_df.set_index('job_offer_id')
    translated_ = pd.DataFrame(read_json_lines(join(ACC_DIR, 'en_skills_ESCO_transformers.jsonl')))
    translated_ar = pd.DataFrame(read_json_lines(join(ACC_DIR, 'en_skills_ESCO_argos.jsonl')))
    translated_['en_skills'] = translated_['en_skills'].apply(lambda x: '|'.join(x))
    translated_['en_skills'] = translated_['en_skills'].str.replace("I'm sorry, but I don't know.", '')
    translated_ar['en_skills'] = translated_ar['en_skills'].apply(lambda x: '|'.join(x))
    translated_ = translated_.set_index('id')
    translated_ar = translated_ar.set_index('id')
    original_df['en_skills'] = translated_['en_skills']
    original_df['labels'] = labels_series
    #original_df['labels'] = original_df['labels'].apply(lambda x: '|'.join(x))
    original_df['scores'] = score_series

    original_df['en_skills_ar'] = translated_ar['en_skills']
    original_df['labels_ar'] = labels_series_ar
    original_df['scores_ar'] = score_series_ar
    #original_df['scores'] = original_df['scores'].apply(lambda x: '|'.join(x))
    return original_df
#         return all_classes



def concat_tables_jo_PE(acc_transformers_df, acc_argos_df):
    labels_series, score_series = process_acceptability_df(acc_transformers_df)
    labels_series_ar, score_series_ar = process_acceptability_df(acc_argos_df)
    original_df = pd.DataFrame(read_json_lines(join(SKILL_OUT, 'extracted_skills_pole_emploi_v2.jsonl')))
    original_df = original_df.set_index('job_offer_id')
    translated_ = pd.DataFrame(read_json_lines(join(ACC_DIR, 'en_skills_PE_transformers.jsonl')))
    translated_ar = pd.DataFrame(read_json_lines(join(ACC_DIR, 'en_skills_pole_emploi_argos.jsonl')))
    translated_['en_skills'] = translated_['en_skills'].apply(lambda x: '|'.join(x))
    translated_['en_skills'] = translated_['en_skills'].str.replace("I'm sorry, but I don't know.", '')
    translated_ar['en_skills'] = translated_ar['en_skills'].apply(lambda x: '|'.join(x))
    translated_ = translated_.set_index('id')
    translated_ar = translated_ar.set_index('id')
    original_df['en_skills'] = translated_['en_skills']
    original_df['labels'] = labels_series
    original_df['en_skills_ar'] = translated_['en_skills']
    original_df['labels_ar'] = labels_series_ar
    #original_df['labels'] = original_df['labels'].apply(lambda x: '|'.join(x))
    original_df['scores'] = score_series


    original_df['en_skills_ar'] = translated_ar['en_skills']
    original_df['labels_ar'] = labels_series_ar
    original_df['scores_ar'] = score_series_ar
    #original_df['scores'] = original_df['scores'].apply(lambda x: '|'.join(x))
    return original_df

def concat_tables_courses_ESCO(acc_transformers_df, acc_argos_df):
    labels_series, score_series = process_acceptability_df(acc_transformers_df)
    labels_series_ar, score_series_ar = process_acceptability_df(acc_argos_df)
    original_df = pd.DataFrame(read_json_lines(join(SKILL_OUT, 'fr_coursera_extracted_skills_v2.jsonl')))
    original_df = original_df.set_index('id')
    translated_ = pd.DataFrame(read_json_lines(join(ACC_DIR, 'en_skills_coursera_transformers.jsonl')))
    translated_ar = pd.DataFrame(read_json_lines(join(ACC_DIR, 'en_skills_coursera_argos.jsonl')))
    translated_['en_skills'] = translated_['en_skills'].apply(lambda x: '|'.join(x))
    translated_['en_skills'] = translated_['en_skills'].str.replace("self-", '')
    translated_ar['en_skills'] = translated_ar['en_skills'].apply(lambda x: '|'.join(x))
    translated_ = translated_.set_index('id')
    translated_ar = translated_ar.set_index('id')
    original_df['en_skills'] = translated_['en_skills']
    original_df['labels'] = labels_series
    original_df['scores'] = score_series
    original_df['en_skills_ar'] = translated_ar['en_skills']
    original_df['labels_ar'] = labels_series_ar
    original_df['scores_ar'] = score_series_ar
    original_df
    #original_df['scores'] = original_df['scores'].apply(lambda x: '|'.join(x))
    return original_df


def concat_tables_courses_PE(acc_transformers_df, acc_argos_df):
    labels_series, score_series = process_acceptability_df(acc_transformers_df)
    labels_series_ar, score_series_ar = process_acceptability_df(acc_argos_df)
    original_df = pd.DataFrame(read_json_lines(join(SKILL_OUT, 'fr_coursera_extracted_skills_pole_emploi_v1.jsonl')))
    original_df = original_df.set_index('id')
    translated_ = pd.DataFrame(read_json_lines(join(ACC_DIR, 'en_skills_coursera_PE_transformers.jsonl')))
    translated_ar = pd.DataFrame(read_json_lines(join(ACC_DIR, 'en_skills_coursera_PE_argos.jsonl')))
    translated_['en_skills'] = translated_['en_skills'].apply(lambda x: '|'.join(x))
    translated_['en_skills'] = translated_['en_skills'].str.replace("self-", '')
    translated_ar['en_skills'] = translated_ar['en_skills'].apply(lambda x: '|'.join(x))
    translated_ = translated_.set_index('id')
    translated_ar = translated_ar.set_index('id')
    original_df['en_skills'] = translated_['en_skills']
    original_df['labels'] = labels_series
    #original_df['labels'] = original_df['labels'].apply(lambda x: '|'.join(x))
    original_df['scores'] = score_series

    original_df['en_skills_ar'] = translated_ar['en_skills']
    original_df['labels_ar'] = labels_series_ar
    original_df['scores_ar'] = score_series_ar
    #original_df['scores'] = original_df['scores'].apply(lambda x: '|'.join(x))
    return original_df

def acceptable_skills(original_df):
  out_data = []
  for ind, row in tqdm(original_df.iterrows(), total = original_df.shape[0]):
    #print(row['en_skills'])
    # if row['en_skills'] == '|':
    #   row['en_skills'] = []
    # else:
    #   row['en_skills'] = row['en_skills'].split('|')
    if row['skills'] == '':
      row['skills'] = ''
    else:
      row['skills'] = row['skills'].split('|')
    acc_list = []
    for x, y, z, a, b in zip(row['skills'], row['labels'], row['scores'], row['labels_ar'], row['scores_ar']):
      if (y == 'LABEL_1')  & (a == 'LABEL_1') & (abs(z-b) < 0.15) & (b > 0.60):
        acc_list.append(x)
    row['acceptable_skills'] = '|'.join(acc_list)
    out_data.append(row)
  return pd.DataFrame(out_data)

def merge_ESCO_PE(df, filename):
  for ind, row in tqdm(df.iterrows(), total = df.shape[0]):
    acceptable_lines = {}
    acceptable_lines['id'] = row['job_offer_id']
    row['acceptable_skills'] = row['acceptable_skills'].split('|')
    row['acceptable_skills_pe'] = row['acceptable_skills_pe'].split('|')
    acceptable_lines['acceptable_skills'] = list(set(row['acceptable_skills']  + row['acceptable_skills_pe']))
    write_json_lines(filename,acceptable_lines)

def merge_ESCO_PE_coursera(df, filename):
  for ind, row in tqdm(df.iterrows(), total = df.shape[0]):
    acceptable_lines = {}
    acceptable_lines['id'] = row['id']
    row['acceptable_skills'] = row['acceptable_skills'].split('|')
    row['acceptable_skills_pe'] = row['acceptable_skills_pe'].split('|')
    acceptable_lines['acceptable_skills'] = list(set(row['acceptable_skills']  + row['acceptable_skills_pe']))
    write_json_lines(filename,acceptable_lines)

## Acceptability Processing: Job offers

In [76]:
accept_df = pd.read_csv(join(ACC_DIR, 'en_skills_ESCO_transformers_raw_acceptability.csv'))
accept_ar_df = pd.read_csv(join(ACC_DIR, 'en_skills_ESCO_argos_raw_acceptability.csv'))

accept_pe_df = pd.read_csv(join(ACC_DIR, 'en_skills_PE_transformers_raw_acceptability.csv'))
accept_pe_ar_df = pd.read_csv(join(ACC_DIR, 'en_skills_pole_emploi_argos_raw_acceptability.csv'))

In [77]:
accept_ar_df.head()

,label,score,id
0,LABEL_1,0.841982,0
1,LABEL_0,0.840149,0
2,LABEL_1,0.916894,0
3,LABEL_1,0.646024,0
4,LABEL_1,0.769747,0


In [78]:
df_ = concat_tables_jo_ESCO(accept_df, accept_ar_df)
df_ = df_.dropna()


In [79]:
df_pe = concat_tables_jo_PE(accept_pe_df, accept_pe_ar_df)
df_pe = df_pe.dropna()

In [80]:
df_.head()

,skills,en_skills,labels,scores,en_skills_ar,labels_ar,scores_ar
job_offer_id,,,,,,,
0,utilisation de matériel|s’|résoudre des incide...,use of equipment|s----------------------------...,"[LABEL_1, LABEL_0, LABEL_1, LABEL_1, LABEL_1, ...","[0.8419815301895142, 0.7044380307197571, 0.815...",use of equipment|s’|resolution of incidents|ha...,"[LABEL_1, LABEL_0, LABEL_1, LABEL_1, LABEL_1, ...","[0.8419815301895142, 0.84014892578125, 0.91689..."
1,,,[LABEL_1],[0.7436407804489136],,[LABEL_1],[0.7436407804489136]
2,,,[LABEL_1],[0.7436407804489136],,[LABEL_1],[0.7436407804489136]
110,,,[LABEL_1],[0.7436407804489136],,[LABEL_1],[0.7436407804489136]
1015,mécanique,mechanical,[LABEL_1],[0.7579735517501831],mechanical,[LABEL_1],[0.7579735517501831]


In [81]:
df = acceptable_skills(df_.reset_index())


  0%|          | 0/19986 [00:00<?, ?it/s]

In [82]:
df.head()

,job_offer_id,skills,en_skills,labels,scores,en_skills_ar,labels_ar,scores_ar,acceptable_skills
0,0,"[utilisation de matériel, s’, résoudre des inc...",use of equipment|s----------------------------...,"[LABEL_1, LABEL_0, LABEL_1, LABEL_1, LABEL_1, ...","[0.8419815301895142, 0.7044380307197571, 0.815...",use of equipment|s’|resolution of incidents|ha...,"[LABEL_1, LABEL_0, LABEL_1, LABEL_1, LABEL_1, ...","[0.8419815301895142, 0.84014892578125, 0.91689...",utilisation de matériel|résoudre des incidents...
1,1,,,[LABEL_1],[0.7436407804489136],,[LABEL_1],[0.7436407804489136],
2,2,,,[LABEL_1],[0.7436407804489136],,[LABEL_1],[0.7436407804489136],
3,110,,,[LABEL_1],[0.7436407804489136],,[LABEL_1],[0.7436407804489136],
4,1015,[mécanique],mechanical,[LABEL_1],[0.7579735517501831],mechanical,[LABEL_1],[0.7579735517501831],mécanique


In [83]:
len(set(sum(list(df['acceptable_skills'].str.split('|')), [])))

25267

In [84]:
22808
20129
25267

25267

In [85]:
df_pe = acceptable_skills(df_pe.reset_index())

  0%|          | 0/116819 [00:00<?, ?it/s]

In [86]:
# df.reset_index(level=0, inplace=True)
# df_pe.reset_index(level=0, inplace=True)

In [87]:
df

,index,job_offer_id,skills,en_skills,labels,scores,en_skills_ar,labels_ar,scores_ar,acceptable_skills
0,0,0,"[utilisation de matériel, s’, résoudre des inc...",use of equipment|s----------------------------...,"[LABEL_1, LABEL_0, LABEL_1, LABEL_1, LABEL_1, ...","[0.8419815301895142, 0.7044380307197571, 0.815...",use of equipment|s’|resolution of incidents|ha...,"[LABEL_1, LABEL_0, LABEL_1, LABEL_1, LABEL_1, ...","[0.8419815301895142, 0.84014892578125, 0.91689...",utilisation de matériel|résoudre des incidents...
1,1,1,,,[LABEL_1],[0.7436407804489136],,[LABEL_1],[0.7436407804489136],
2,2,2,,,[LABEL_1],[0.7436407804489136],,[LABEL_1],[0.7436407804489136],
3,3,110,,,[LABEL_1],[0.7436407804489136],,[LABEL_1],[0.7436407804489136],
4,4,1015,[mécanique],mechanical,[LABEL_1],[0.7579735517501831],mechanical,[LABEL_1],[0.7579735517501831],mécanique
...,...,...,...,...,...,...,...,...,...,...
19981,19981,33599,"[travailler en équipe, veiller au respect du r...",working as a team|ensuring compliance with the...,"[LABEL_1, LABEL_1, LABEL_0, LABEL_1, LABEL_1, ...","[0.8828555345535278, 0.9373631477355956, 0.952...",working as a team|ensuring compliance|manage t...,"[LABEL_1, LABEL_1, LABEL_1, LABEL_1, LABEL_1, ...","[0.8828555345535278, 0.8826057314872742, 0.935...",travailler en équipe|veiller au respect du règ...
19982,19982,33600,"[fixer, organiser les déplacements du responsa...",fix|arrange the travel of the person in charge...,"[LABEL_1, LABEL_1, LABEL_1, LABEL_1, LABEL_1, ...","[0.6585937738418579, 0.8911353349685669, 0.760...",fix|organize the movement of the official|form...,"[LABEL_1, LABEL_1, LABEL_0, LABEL_1, LABEL_1, ...","[0.6585937738418579, 0.8675668239593506, 0.545...",fixer|organiser les déplacements du responsabl...
19983,19983,33601,[envoyer des reportings réguliers],send regular reports,[LABEL_1],[0.8915622234344482],send regular reporting,[LABEL_1],[0.9319383502006532],envoyer des reportings réguliers
19984,19984,33602,"[bonne connaissance, assurer une veille concur...",good knowledge|ensuring a competitive watch.de...,"[LABEL_0, LABEL_0, LABEL_1, LABEL_1, LABEL_1, ...","[0.5611529350280762, 0.8431794047355652, 0.938...",knowledge|ensure a competitive watch.de|ensure...,"[LABEL_1, LABEL_1, LABEL_1, LABEL_1, LABEL_1, ...","[0.5371708869934082, 0.5569312572479248, 0.864...",veiller à la réalisation de vos objectifs|assu...


In [89]:
df__= df[['job_offer_id', 'acceptable_skills']]
df__pe = df_pe[['job_offer_id', 'acceptable_skills']]

In [90]:
df__.set_index('job_offer_id')
df__pe.set_index('job_offer_id')

,acceptable_skills
job_offer_id,
0,utilisation de matériel|assistance technique|g...
2,
1015,gestion de production|mécanique
1016,conduite d'opérations
1017,informatique de gestion|gérer différentes tâches
...,...
52062,méthodes de gestion|assurer la surveillance de...
51878,techniques de vente|normes et standards|assure...
51877,logiciel professionnel d'exploitation de trans...


In [91]:
df__['acceptable_skills_pe'] = df__pe['acceptable_skills']

In [92]:
merge_ESCO_PE(df__, join(ACC_DIR, 'filtered_hard_skills_jobs.jsonl'))

  0%|          | 0/19986 [00:00<?, ?it/s]

In [93]:
d  = pd.DataFrame(read_json_lines(join(ACC_DIR, 'filtered_hard_skills_jobs.jsonl')))

In [94]:
#### Done
d

,id,acceptable_skills
0,0,"[matérielles, logicielles, prendre en compte, ..."
1,1,[]
2,2,"[, gestion de production]"
3,110,[]
4,1015,"[, gérer différentes tâches, informatique de g..."
...,...,...
39967,33599,"[outils pédagogiques, l’organisation, assurer ..."
39968,33600,"[fixer, gérer le déplacement des outils, opéra..."
39969,33601,[participer à des missions à caractère académi...
39970,33602,"[veiller à la réalisation de vos objectifs, co..."


## Acceptability: Processing Courses

In [95]:
accept_df = pd.read_csv(join(ACC_DIR, 'en_skills_coursera_transformers_raw_acceptability.csv'))
accept_ar_df = pd.read_csv(join(ACC_DIR, 'en_skills_coursera_argos_raw_acceptability.csv'))

accept_pe_df = pd.read_csv(join(ACC_DIR, 'en_skills_coursera_PE_transformers_raw_acceptability.csv'))
accept_pe_ar_df = pd.read_csv(join(ACC_DIR, 'en_skills_coursera_PE_argos_raw_acceptability.csv'))

In [96]:
df_ = concat_tables_courses_ESCO(accept_df, accept_ar_df)
df_ = df_.dropna()


df_pe = concat_tables_courses_PE(accept_pe_df, accept_pe_ar_df)
df_pe = df_pe.dropna()

In [97]:
df = acceptable_skills(df_.reset_index())
df_pe = acceptable_skills(df_pe.reset_index())

  0%|          | 0/1271 [00:00<?, ?it/s]

  0%|          | 0/1721 [00:00<?, ?it/s]

In [98]:
# df.reset_index(level=0, inplace=True)
# df_pe.reset_index(level=0, inplace=True)

In [99]:
df__= df[['id', 'acceptable_skills']]
df__pe = df_pe[['id', 'acceptable_skills']]

In [100]:
df__.set_index('id')
df__pe.set_index('id')

,acceptable_skills
id,
DMkcgX7LEeyRTg6FtAvfBw,différents types|intégrer des applications|tab...
69Bku0KoEeWZtA4u62x6lQ,business
zRhFMLBVEeqP6QqV6TddSw,business
7zPEd7ErEeyiqQr1qP1QDw,business
3uRPpdGFEeiElA6q2-9yxA,leadership
...,...
ESRGuuYzEeeUDhLeFYRShg,business
sfYP0IEsEeyWPQqrmHZ9QQ,business
Zz0lopT9EeyHuA5cIqn4NQ,tester des composants|ecrire des tests


In [101]:
df__['acceptable_skills_pe'] = df__pe['acceptable_skills']

In [102]:
merge_ESCO_PE_coursera(df__, join(ACC_DIR, 'filtered_hard_skills_coursera.jsonl'))

  0%|          | 0/1271 [00:00<?, ?it/s]

In [103]:
d  = pd.DataFrame(read_json_lines(join(ACC_DIR, 'filtered_hard_skills_coursera.jsonl')))

In [104]:
d

,id,acceptable_skills
0,NJSdGN71Eeq4CApSN3OTvQ,"[différents types, intégrer des applications, c#]"
1,DMkcgX7LEeyRTg6FtAvfBw,"[différents types, , compétences de base]"
2,rrZX06kxEeyjPxJkzIUqCw,[]
3,B6LZjKkwEeyjPxJkzIUqCw,[]
4,RdmZEakyEeyjPxJkzIUqCw,[]
...,...,...
2537,04e4_kIxEeuVrwrlQzvd6w,"[business, principes de base]"
2538,N7ghLqkpEeyjPxJkzIUqCw,"[, business]"
2539,Zz0lopT9EeyHuA5cIqn4NQ,"[business, tester des composants]"
2540,3bL_2qkxEeyjPxJkzIUqCw,"[, créer des bases de données spatiales, théor..."


## Compare numbers of skills before and after filtering

In [ ]:
dataf = pd.DataFrame(read_json_lines(join(SKILL_OUT, 'fr_extracted_skills_v3.jsonl')))
dataf_pe = pd.DataFrame(read_json_lines(join(SKILL_OUT, 'extracted_skills_pole_emploi_v2.jsonl')))

filtered_data =  pd.DataFrame(read_json_lines(join(ACC_DIR, 'filtered_hard_skills_jobs.jsonl')))

In [ ]:
filtered_data.explode('acceptable_skills')

,id,acceptable_skills
0,0,"matérielles, logicielles"
0,0,prendre en compte
0,0,environnement de travail
0,0,systèmes d’exploitation
0,0,utilisation de matériel
...,...,...
19985,33603,support technique
19985,33603,location de véhicules
19985,33603,bases de données
19985,33603,base de données


In [ ]:
dataf['skills'] = dataf['skills'].apply(lambda x: x.split('|'))
dataf.explode('skills')

,job_offer_id,skills
0,0,utilisation de matériel
0,0,s’
0,0,résoudre des incidents
0,0,architectures matérielles
0,0,environnement de travail
...,...,...
123990,51718,plan technique
123990,51718,respecter les engagements qualité dans le cadr...
123990,51718,mise à jour
123990,51718,problèmes techniques


In [ ]:
dataf_pe['skills'] = dataf_pe['skills'].apply(lambda x: x.split('|'))
dataf_pe.explode('skills')

,job_offer_id,skills
0,0,chargé de clientèle
0,0,utilisation de matériel
0,0,assistance technique
0,0,gérer des demandes des utilisateurs par téléphone
0,0,prendre en charge
...,...,...
123990,51718,application des procédures
123990,51718,problèmes techniques
123990,51718,mise à jour
123990,51718,normes de sécurité


## Extract skills from ESCO

In [ ]:
def read_ESCO_files():
  skills = []
  all_ESCO_files = ['digCompSkillsCollection_fr.csv', 'digitalSkillsCollection_fr.csv', 'greenSkillsCollection_fr.csv', 'languageSkillsCollection_fr.csv', 
                    'researchSkillsCollection_fr.csv', 'skills_fr.csv', 'transversalSkillsCollection_fr.csv' ]
  for i in all_ESCO_files:
   df = pd.read_csv(join(SKILL_DB_DIR, 'ESCO', i))
   skills.append(df['preferredLabel'].tolist())
  all_skills = [item for sublist in skills for item in sublist]
  return all_skills

In [ ]:
all_skills_ESCO = read_ESCO_files()

## Extract skills from pole emploi

In [ ]:
import re

def split_skills(skill_concat):
    if isinstance(skill_concat, str):
        #skill_concat = skill_concat.replace("'", ' ')
        skill_list = re.split(",(?=[A-Z])", skill_concat)
        skill_list = [i for i in skill_list if len(i)>0]
        #re.sub("(^|[.?!])\s*([a-zA-Z])", lambda p: p.group(0).upper(), s)
        skill_list = [(i[:-1] if (i[-1]==',' ) else i).lower() for i in skill_list]
        if 'l' in skill_list:
            print(skill_concat)
        return skill_list
    else:
        return ['']

def get_all_pole_emploi_skills():
#     savoir_faire_base 	savoirs_base 	savoir_faire_specifiques 	savoirs_specifiques
    all_skills = []
    df = pd.read_csv(join(SKILL_DB_DIR, 'pole_emploi_complete.csv'))
    all_skills += sum([split_skills(i) for i in list(df['savoir_faire_base'])],[])
    all_skills += sum([split_skills(i) for i in list(df['savoirs_base'])],[])
    all_skills += sum([split_skills(i)  for i in list(df['savoir_faire_specifiques'])],[])
    all_skills += sum([split_skills(i)  for i in list(df['savoirs_specifiques'])],[])
    
    all_skills = [i.strip().lower() for i in all_skills if len(i)>0]
    return all_skills

In [ ]:
all_skills_pe = get_all_pole_emploi_skills()

In [ ]:
all_skills_pe

In [ ]:
[i for i in all_skills_pe if i =='e']

[]

In [ ]:
all_skills = all_skills_ESCO + all_skills_pe

In [ ]:
len(all_skills)

40981

In [ ]:
all_skills = list(set(all_skills))

In [ ]:
len(all_skills)

24887

In [ ]:
list(pd.read_csv(join(SKILL_DB_DIR, 'pole_emploi_complete.csv'))['savoir_faire_base'])[0]

"Préparer le matériel, les matériaux et les outillages,Identifier le type d'intervention,Sécuriser un équipement,Atteler un équipement,Conduire les engins agricoles vers le lieu de production (champs, forêt), de stockage, de livraison (fermes, coopératives),Matérialiser une implantation de parcelle ou de passage,Approvisionner des engins d'exploitation,Réaliser une opération de maintenance"

In [ ]:
split_skills(skill_concat = list(pd.read_csv(join(SKILL_DB_DIR, 'pole_emploi_complete.csv'))['savoir_faire_base'])[0])

['préparer le matériel, les matériaux et les outillages',
 "identifier le type d'intervention",
 'sécuriser un équipement',
 'atteler un équipement',
 'conduire les engins agricoles vers le lieu de production (champs, forêt), de stockage, de livraison (fermes, coopératives)',
 'matérialiser une implantation de parcelle ou de passage',
 "approvisionner des engins d'exploitation",
 'réaliser une opération de maintenance']

## Rekrute  Job offers 

In [ ]:
jo_df = pd.read_csv(join(JO_OUT, 'all_job_offers_clean.csv'))

In [ ]:
jo_df['Poste'] = jo_df['Poste'].replace(np.nan, '')
jo_df['Profil recherché'] = jo_df['Profil recherché'].replace(np.nan, '')
jo_df['Qualités requises pour réussir dans ce rôle'] = jo_df['Qualités requises pour réussir dans ce rôle'].replace(np.nan, '')

In [ ]:
jo_df['post_data'] = jo_df['Poste']+jo_df['Profil recherché'] + jo_df['Qualités requises pour réussir dans ce rôle']

In [ ]:
jo_df['post_data'] = jo_df['post_data'].str.lower()
jo_df['post_data'] = jo_df['post_data'].str.replace('’',"'")

In [ ]:
jo_skills = jo_df['post_data'].tolist()

## Compare using fuzzy

In [ ]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
skills_fuzzy = []
#process.extract(all_skills[0], jo_skills[0])
fuzz.ratio(jo_skills[3], all_skills[0])

2

In [ ]:
ss = [all_skills[0], all_skills[1], all_skills[2]]
skill = []

In [ ]:
rl = process.extract(jo_skills[0],ss)
for index, tup in enumerate(rl):
  if tup[1] > 90:
    skill.append(tup[0])

In [ ]:
skill

[]

In [ ]:
 #[('New York Jets', 100), ('New York Giants', 78)]
    #if fuzz.partial_ratio(row['post_data'], i) > 90:
      #skill.append(i)
  #skills.append('|'.join(skill))
  #print(skill)

In [ ]:
skills = []
for ind, row in tqdm(jo_df.iterrows(), total=jo_df.shape[0]):
  skill = []
  ratios_list = process.extract(row['post_data'], all_skills)
  for index, tup in enumerate(ratios_list):
    if tup[1] > 90:
      skill.append(tup[0])
  skills.append('|'.join(skill))
  #print(row['post_data'])
  print(skills)
jo_df['skills'] = skills

In [ ]:
skills = []
for ind, row in tqdm(jo_df.iterrows(), total=jo_df.shape[0]):
  skill = []
  #print(row['post_data'])
  for i in  all_skills:
    for j in row['post_data'].split('.'):
      if int(fuzz.partial_ratio(j, i)) > 90:
        skill.append(i)
        print(i)
  skills.append('|'.join(skill))
  #print(row['post_data'])
  #print(skill)
jo_df['skills'] = skills

In [ ]:
import textdistance

In [ ]:
for i in tst.split('.'):
  #print(i)
  print(fuzz.partial_ratio(i, "complète est assurée"))
  print(textdistance.LCSStr()(i, "complète est assurée"))
  #print(type(textdistance.LCSStr()(i, "complète est assurée")))

In [ ]:
fuzz.token_set_ratio(tst, "les requêtes d'utilisateurs")

In [ ]:
tst.find("connaissance générale")

In [ ]:
[i for i in all_skills if i =='informatique']

['informatique',
 'informatique',
 'informatique',
 'informatique',
 'informatique',
 'informatique',
 'informatique',
 'informatique',
 'informatique',
 'informatique',
 'informatique',
 'informatique']

In [ ]:
[i for i in all_skills if i =='q']

In [ ]:
for i in jo_skills:
  for j in all_skills:
    fuzz.ratio(i, j)
    ratios.append()

In [ ]:
all_skills[0]

'résoudre des problèmes techniques'

In [ ]:
jo_skills[0]

"le chargé de clientèle en assistance technique a la principale mission de recueillir puis traiter à distance les requêtes d'utilisateurs en difficulté au regard d'une utilisation de matériel informatique. rattaché au responsable d'équipe, la mission consiste à : gérer des demandes des utilisateurs par téléphone de manière optimale diagnostiquer des problèmes système et réseau prendre en compte et résoudre des incidents traiter les demandes de changement en provenance des utilisateurs avant de prendre en charge votre mission, une formation complète est assurée afin de vous préparer aux responsabilités qui vous seront confiées.compétences: parfaite maîtrise de la langue française, avec une bonne aisance à l’oral. sens de l’écoute et de l’orientation. bonne capacité de diagnostic et d'analyse. connaissance générale des concepts de base et des techniques d’architecture des systèmes et des réseaux. connaissance générale des différents systèmes d’exploitation usuels, des architectures matér

## Old stuff

In [ ]:
def clean_skills(skill):
    doc = fr_nlp(skill)
    tokens = [token for token in doc]
    tokens_pos = [k.pos_ for k in tokens]
    tokens_tag = [k.tag_ for k in tokens]
    tokens_dep = [k.dep_ for k in tokens]
    return tokens_pos, tokens_tag, tokens_dep

In [ ]:
def is_clean(skill):
  doc = fr_nlp(skill)
  tokens = [token for token in doc]
  if len(tokens) == 0:
    return False
  if tokens[len(tokens)-1].pos_ in ["VERB", "PUNCT"] or tokens[0].text in ['sein', 'ressources', 'biens'] or tokens[len(tokens)-1].text in ['terme', 'l’aspect', 'connaissance', 'touts', "'", "sein"]:
    return False
  else:
    return True

In [ ]:
def get_clean_skills(df):
  clean_sentences = []
  for ind, row  in tqdm(df.iterrows(), total=df.shape[0]):
    sentences = (row['skills'].split('|'))
    for sent in sentences:
      if (is_clean(sent)):
        clean_sentences.append(sent)
        row['skills'] = clean_sentences
  return df

In [ ]:
get_clean_skills(df)

In [ ]:
is_clean("s'")

False

In [ ]:
#['amod', 'ROOT']
#['ROOT', 'case', 'det', 'nmod']   clean
#ROOT', 'case', 'nmod]
#['ROOT', 'amod']
#['ROOT', 'obj']

In [ ]:
clean_skills("gestion administrative")

(['NOUN', 'ADJ'], ['NOUN', 'ADJ'], ['ROOT', 'amod'])

In [ ]:
sentence  = "aider les approvisionnements à suivre préparer les plans d'actions|suivre les plans d'actions|évaluer les fournisseurs.de formation|management de la qualité"

In [ ]:
sentence.split('|')

["aider les approvisionnements à suivre préparer les plans d'actions",
 "suivre les plans d'actions",
 'évaluer les fournisseurs.de formation',
 'management de la qualité']